# Algorithm 21: rigidFrom3Points

This algorithm constructs a rigid body transformation (rotation + translation) from three points that define a reference frame. In protein structure, this is typically used with backbone atoms N, CA, C to define the local coordinate frame of each residue.

## Algorithm Pseudocode

![rigidFrom3Points](../imgs/algorithms/rigidFrom3Points.png)

## Source Code Location
- **File**: `AF2-source-code/model/r3.py`
- **Function**: `rigids_from_3_points`
- **Lines**: 78-110

## Geometric Construction

Given three points (x1, x2, x3), we construct an orthonormal frame:

1. **Origin**: x2 (typically CA atom)
2. **v1**: Normalized vector from x2 to x1 (x1 - x2)
3. **v2**: Vector from x2 to x3 (x3 - x2)
4. **e1**: v1 normalized (first basis vector)
5. **u2**: v2 - (v2·e1)e1 (component of v2 orthogonal to e1)
6. **e2**: u2 normalized (second basis vector)
7. **e3**: e1 × e2 (third basis vector, cross product)

In [ ]:
import numpy as np

np.random.seed(42)

## NumPy Implementation

In [ ]:
def rigid_from_3_points(x1, x2, x3, eps=1e-8):
    """
    Construct a rigid transformation from three points.
    
    Algorithm 21 from AlphaFold2 supplementary materials.
    
    The resulting frame has:
    - Origin at x2
    - x-axis pointing from x2 toward x1
    - y-axis in the plane of x1, x2, x3
    - z-axis perpendicular to the plane
    
    For protein backbone:
    - x1 = N atom position
    - x2 = CA atom position (origin)
    - x3 = C atom position
    
    Args:
        x1: First point [N, 3] or [3]
        x2: Second point (origin) [N, 3] or [3]
        x3: Third point [N, 3] or [3]
        eps: Small value for numerical stability
    
    Returns:
        rotation: Rotation matrix [N, 3, 3] or [3, 3]
        translation: Translation vector [N, 3] or [3] (same as x2)
    """
    # Handle both single and batched inputs
    single_input = len(x1.shape) == 1
    if single_input:
        x1 = x1[None, :]
        x2 = x2[None, :]
        x3 = x3[None, :]
    
    N = x1.shape[0]
    
    # Step 1: Compute vectors from origin (x2)
    # Line 1: v1 = x1 - x2
    v1 = x1 - x2
    
    # Line 2: v2 = x3 - x2
    v2 = x3 - x2
    
    print(f"v1 (N - CA direction): {v1[0]}")
    print(f"v2 (C - CA direction): {v2[0]}")
    
    # Step 2: Construct first basis vector (normalized v1)
    # Line 3: e1 = v1 / ||v1||
    e1 = v1 / (np.linalg.norm(v1, axis=-1, keepdims=True) + eps)
    
    # Step 3: Gram-Schmidt orthogonalization
    # Line 4: u2 = v2 - (v2 · e1) * e1
    # This removes the component of v2 parallel to e1
    dot_v2_e1 = np.sum(v2 * e1, axis=-1, keepdims=True)
    u2 = v2 - dot_v2_e1 * e1
    
    # Line 5: e2 = u2 / ||u2||
    e2 = u2 / (np.linalg.norm(u2, axis=-1, keepdims=True) + eps)
    
    # Step 4: Third basis vector via cross product
    # Line 6: e3 = e1 × e2
    e3 = np.cross(e1, e2)
    
    print(f"e1 (x-axis): {e1[0]}")
    print(f"e2 (y-axis): {e2[0]}")
    print(f"e3 (z-axis): {e3[0]}")
    
    # Step 5: Construct rotation matrix
    # Columns are the basis vectors
    rotation = np.stack([e1, e2, e3], axis=-1)  # [N, 3, 3]
    
    # Translation is just the origin (x2)
    translation = x2.copy()
    
    if single_input:
        rotation = rotation[0]
        translation = translation[0]
    
    return rotation, translation

## Test with Protein Backbone Geometry

In [ ]:
# Standard backbone geometry (approximate values in Angstroms)
# These are relative positions in the local frame

# Ideal backbone bond lengths and angles
N_CA_length = 1.46  # Angstroms
CA_C_length = 1.52  # Angstroms
N_CA_C_angle = np.radians(111)  # degrees -> radians

# Place atoms in a simple configuration
# CA at origin
CA = np.array([0.0, 0.0, 0.0])

# N along negative x-axis
N = np.array([-N_CA_length, 0.0, 0.0])

# C in the xy-plane at appropriate angle
C = np.array([
    CA_C_length * np.cos(np.pi - N_CA_C_angle),
    CA_C_length * np.sin(np.pi - N_CA_C_angle),
    0.0
])

print("Standard backbone geometry:")
print(f"  N:  {N}")
print(f"  CA: {CA}")
print(f"  C:  {C}")
print(f"  N-CA distance: {np.linalg.norm(N - CA):.3f} Å")
print(f"  CA-C distance: {np.linalg.norm(C - CA):.3f} Å")

In [ ]:
# Construct frame from N, CA, C
rotation, translation = rigid_from_3_points(N, CA, C)

print(f"\nRotation matrix:")
print(rotation)
print(f"\nTranslation (CA position):")
print(translation)

## Verify Orthonormality

In [ ]:
# Check that rotation matrix is orthonormal
# R @ R^T should be identity
R_RT = rotation @ rotation.T
print("R @ R^T (should be identity):")
print(R_RT)

# Determinant should be +1 (proper rotation)
det = np.linalg.det(rotation)
print(f"\nDeterminant: {det:.6f} (should be 1.0)")

# Check orthogonality of columns
e1, e2, e3 = rotation[:, 0], rotation[:, 1], rotation[:, 2]
print(f"\nDot products (should be ~0):")
print(f"  e1 · e2 = {np.dot(e1, e2):.6f}")
print(f"  e1 · e3 = {np.dot(e1, e3):.6f}")
print(f"  e2 · e3 = {np.dot(e2, e3):.6f}")

## Batched Example

In [ ]:
# Create multiple residue frames
N_residues = 10

# Random but plausible backbone positions
# Start from a straight chain and add some noise
CA_positions = np.zeros((N_residues, 3))
for i in range(N_residues):
    CA_positions[i] = [i * 3.8, 0, 0]  # ~3.8 Å CA-CA distance

# Add some randomness
CA_positions += np.random.randn(N_residues, 3) * 0.5

# Generate N and C positions relative to CA
N_positions = CA_positions + np.array([-1.46, 0, 0]) + np.random.randn(N_residues, 3) * 0.1
C_positions = CA_positions + np.array([1.0, 1.1, 0]) + np.random.randn(N_residues, 3) * 0.1

print(f"Generated {N_residues} residue backbone frames")
print(f"N positions shape: {N_positions.shape}")
print(f"CA positions shape: {CA_positions.shape}")
print(f"C positions shape: {C_positions.shape}")

In [ ]:
# Compute frames for all residues
rotations, translations = rigid_from_3_points(N_positions, CA_positions, C_positions)

print(f"\nOutput shapes:")
print(f"  Rotations: {rotations.shape}")
print(f"  Translations: {translations.shape}")

# Verify all are proper rotations
dets = np.array([np.linalg.det(r) for r in rotations])
print(f"\nDeterminants: {dets}")
print(f"All close to 1.0: {np.allclose(dets, 1.0)}")

## Using Frames to Transform Points

In [ ]:
def apply_frame(rotation, translation, points_local):
    """Transform points from local to global frame."""
    return np.einsum('...ij,...j->...i', rotation, points_local) + translation

def inverse_frame(rotation, translation, points_global):
    """Transform points from global to local frame."""
    return np.einsum('...ij,...j->...i', rotation.swapaxes(-1, -2), points_global - translation)

# Test: transform local frame points to global
# In local frame, CB is approximately at:
CB_local = np.array([-0.5, -0.8, -1.2])  # Approximate CB position in local frame

# Transform to global positions for each residue
CB_global = apply_frame(rotations, translations, CB_local)

print(f"CB positions in global frame:")
for i in range(3):
    print(f"  Residue {i}: {CB_global[i]}")

# Verify round-trip
CB_back = inverse_frame(rotations, translations, CB_global)
print(f"\nRound-trip error: {np.abs(CB_back - CB_local).max():.6f}")

## Source Code Reference

```python
# From AF2-source-code/model/r3.py

def rigids_from_3_points(
    point_on_neg_x_axis: Vecs,  # N atom
    origin: Vecs,                # CA atom  
    point_on_xy_plane: Vecs,     # C atom
) -> Rigids:
  """Create Rigids from 3 points.

  Jumper et al. (2021) Suppl. Alg. 21 "rigidFrom3Points"
  """
  m = rots_from_two_vecs(
      e0_unnormalized=vecs_sub(origin, point_on_neg_x_axis),
      e1_unnormalized=vecs_sub(point_on_xy_plane, origin))

  return Rigids(rot=m, trans=origin)
```

## Key Insights

1. **Gram-Schmidt**: The algorithm uses Gram-Schmidt orthogonalization to ensure the resulting frame is orthonormal.

2. **Right-Handed**: The cross product ensures a right-handed coordinate system.

3. **Backbone Convention**: For protein backbones, N→CA defines the x-axis, with C determining the xy-plane.

4. **Invariant Representation**: These frames enable SE(3)-invariant processing in the structure module.